In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import json
import csv
import re

In [ ]:
laptop_page_url = "https://tiki.vn/api/v2/products?limit=48&include=advertisement&aggregations=1&category=8322&page={}&urlKey=nha-sach-tiki"
product_url = "https://tiki.vn/api/v2/products/{}"

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}


def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        
        response = requests.get(laptop_page_url.format(i), headers=headers)

        if (response.status_code != 200):
            break

        products = json.loads(response.text)["data"]
        
        if (len(products) == 0):
            break
        print("Crawl page: ", i)
        print(laptop_page_url.format(i))
        for product in products:
            product_id = str(product["id"])
            print("Product ID: ", product_id)
            product_list.append(product_id)

        i += 1
    return product_list, i-1


def save_product_id(product_list=[], product_id_file="product_ids.txt"):
    try:
        with open(product_id_file, "w+") as file:
            data = "\n".join(product_list)
            file.write(data)
        print("File saved as:", product_id_file)
    except Exception as e:
        print("An error occurred:", str(e))

def crawl_product(product_list=[]):
    product_detail_list = []
    for product_id in product_list:
        response = requests.get(product_url.format(product_id), headers=headers)
        if (response.status_code == 200):
            product_detail_list.append(response.text)
            print("Crawl product: ", product_id, ": ", response.status_code)
    return product_detail_list


def save_raw_product(product_detail_list=[], product_data_file="product_data.txt"):
    try:
        with open(product_data_file, "w+") as file:
            data = "\n".join(product_detail_list)
            file.write(data)
        print("File saved as:", product_data_file)
    except Exception as e:
        print("An error occurred:", str(e))
def load_raw_product():
    file = open('product_data.txt', "r")
    return file.readlines()


### Crawl product id


In [ ]:
product_list, page = crawl_product_id()

print("No. Page: ", page)
print("No. Product ID: ", len(product_list))

In [ ]:
save_product_id(product_list)


### Crawl detail for each product id


In [ ]:
product_list = crawl_product(product_list)

In [ ]:
# save product detail for backup
save_raw_product(product_list)

In [ ]:
for product_id in product_list:
    product_data = json.loads(product_id)
    print(product_data)

In [ ]:
product_id_=[]
name_=[]
url_=[]
seller_=[]
price_=[]
original_price_=[]
discount_=[]
discount_rate_=[]
rating_average_=[]
review_count_=[]
primary_category_name_=[]
quantity_sold_=[]
has_ebook_=[]
type_=[]
book_cover_=[]
short_description_=[]
favourite_count_=[]
has_buynow_=[]
author_name_=[]
is_authentic_=[]
publisher_vn_=[]
publication_date_=[]
dimensions_=[]
translators_=[]
number_of_page_=[]
gift_item_title_=[]
stock_item_=[]
categories_=[]

In [ ]:
for product in product_list:
    try:
        product_data = json.loads(product)
        # Có ebook hay không
        has_ebook = product_data.get("has_ebook",'')
        has_ebook_.append(has_ebook)
        # Thể loại
        type = product_data.get("type",'')
        type_.append(type)
        # Tác giả
        authors=product_data.get("authors",[])
        author=[]
        if authors:
            for au in authors:
                author.append(au.get("name",''))
            author_name_.append(', '.join(author))
        else:
            author_name_.append("")
       
        specifications = product_data.get("specifications",[])
        attributes = specifications[0].get("attributes",[])

        au=0
        pu1=0
        pu2=0
        di=0
        tr=0
        nu=0
        for attribute in attributes:
            if attribute.get("code",'')=="is_authentic":
                is_authentic=attribute.get("value",'')
                is_authentic_.append(is_authentic)
                au=1
            if attribute.get("code",'')=="publisher_vn":
                publisher_vn=attribute.get("value",'')
                publisher_vn_.append(publisher_vn)
                pu1=1
        

            if attribute.get("code",'')=="publication_date":
                publication_date=attribute.get("value",'')
                publication_date_.append(publication_date)
                pu2=1

            if attribute.get("code",'')=="dimensions":
                dimensions=attribute.get("value",'')
                dimensions_.append(dimensions)
                di=1

            if attribute.get("code",'')=="dich_gia":
                translators=attribute.get("value",'')
                translators_.append(translators)
                tr=1

            if attribute.get("code",'')=="number_of_page":
                number_of_page=attribute.get("value",'')
                number_of_page_.append(number_of_page)
                nu=1

        if(au==0):
            is_authentic_.append("")
        if(pu1==0):
            publisher_vn_.append("")
        if(pu2==0):
            publication_date_.append("")    
        if(di==0):
            dimensions_.append("")
        if(tr==0):
            translators_.append("")
        if(nu==0):
            number_of_page_.append("")

        # Quà đi kèm
        gift_item_title=product_data.get("gift_item_title",'')
        gift_item_title_.append(gift_item_title)

        # Số sách còn 
        stock_item=product_data.get("stock_item",{})
        stock_item=stock_item.get("qty",'')
        stock_item_.append(stock_item)

        short_description = product_data.get("short_description",'')
        short_description_.append(short_description)

        favourite_count = product_data.get("favourite_count",'')
        favourite_count_.append(favourite_count)

        has_buynow = product_data.get("has_buynow",'')
        if has_buynow:
            has_buynow_.append("Yes")
        else:
            has_buynow_.append("No")

        id = product_data.get("id",'')
        product_id_.append(id)

        name = product_data.get("name", '')
        name_.append(name)
        url ='https://tiki.vn/'+ product_data.get("url_path",'')
        url_.append(url)

        # Đơn vị đang bán
        seller = product_data.get("current_seller",{})
        seller = seller.get("name",'')
        seller_.append(seller)

        price = product_data.get("price",'')
        price_.append(price)

        original_price = product_data.get("original_price",'')
        original_price_.append(original_price)

        discount = product_data.get("discount",'')
        discount_.append(discount)

        discount_rate = product_data.get("discount_rate",'')
        discount_rate_.append(discount_rate)

        rating_average= product_data.get("rating_average",'')
        rating_average_.append(rating_average)

        review_count=product_data.get("review_count",'')
        review_count_.append(review_count)

        primary_category_name=product_data.get("primary_category_name",'')
        primary_category_name_.append(primary_category_name)

        quantity_sold=product_data.get("quantity_sold",{})
        quantity_sold=quantity_sold.get("value",'')
        quantity_sold_.append(quantity_sold)
        
        categorie=product_data.get("categories",{})
        categorie=categorie.get("name",'')
        categories_.append(categorie)
    except:
        continue


### Aggregate data

In [ ]:
df = pd.DataFrame({
    "product_id_": product_id_,
    "name": name_,
    "url": url_,
    "store": seller_,
    "price": price_,
    "original_price": original_price_,
    "discount": discount_,
    "discount_rate": discount_rate_,
    "rating": rating_average_,
    "review_count": review_count_,
    "type": primary_category_name_,
    "quantity_sold": quantity_sold_,
    "has_ebook": has_ebook_,
    "type": type_,
    "author_name":author_name_,
    "short_description": short_description_,
    "favourite_count": favourite_count_,
    "has_buynow": has_buynow_,
    "author_name": author_name_,
    "is_authentic": is_authentic_,
    "publisher_vn_":publisher_vn_,
    "publication_date": publication_date_,
    "dimensions": dimensions_,
    "translators": translators_,
    "number_of_page": number_of_page_,
    "gift_item_title": gift_item_title_,
    "stock_item":stock_item_,
    "categories": categories_
})
df.head(5)

### Save data

In [ ]:
df.to_excel('Data_Tiki.xlsx')
df.to_csv('Data_Tiki.csv')